# Database Support

## APIs and usage

## TODO

## Make this file be able to import other .ipynb files

In [1]:
from IMP import *

## Helper defining 

In [73]:
Docstring = '''
Database relative functional module
'''

import tools as tl
import sqlite3

class cached_property(object):
    ''' A property that is only computed once per instance and then replaces
        itself with an ordinary attribute. Deleting the attribute resets the
        property. '''
        
    def __init__(self, func):
        self.__doc__ = func.__doc__
        self.func = func
    
    def __get__(self, obj, cls):
        if obj is None: return self
        value = obj.__dict__[self.func.__name__] = self.func(obj)
        return value
    

def sqlexec(db, *sql_val):
    cur = db.cursor()
    cur.execute(*sql_val)
    cur.close()
    

class ConfigDB(object):
    @cached_property
    def db(self):
        print('connecting config.db')
        return sqlite3.connect('config.db')
    
    def __init__(self, func):
        self.func = func
        self.__dict__['__doc__'] = func.__doc__
    
    def __call__(self, **argv):
        sql = self.func(**argv)
        cur = self.db.cursor()
        if isinstance(sql, str):
            cur.execute(sql)
        elif isinstance(sql, tuple) and len(sql) == 2:
            cur.execute(*sql)
        else:
            raise(TypeError('sql is not a string or (string, value): %s'%sql))
    
@ConfigDB    
def add_config(sn, addr_info, regs_list, reg_names, info):
    '''
    def add_config(sn, addr_info, regs_list, reg_names, info):
        ...
        
    sn: serial number of xstream
    addr_info: ip@port or address@port
    regs_list: list of registers, from small to large, split by comma(","), eg: 10-20,25,30-32
    reg_names: names of registers, like CO2, H2, Temp, CH4, ch_1, ch_2... numbers of names in reg_names and registers in reg_list MUST BE matched
    info: some description, caption, infomation
    '''    
    args_dict = locals()
    addr, port = tl.get_addr_port(addr_info)
    regs = tl.get_regs_list(regs_list)
    names = tl.get_reg_names(reg_names)
    
    try:
        assert(len(regs) == len(names))
    except AssertionError as e:
        print('numbers of names in reg_names and registers in reg_list MUST BE matched')
        print('regs num: %d, names num: %d'% (len(regs), len(names)))
        raise(tl.PrintColorfulTraceBack)
        
    sql = ('INSERT INTO config(%s,%s,%s,%s,%s) VALUES(?,?,?,?,?)'%tuple(args_dict.keys()), list(args_dict.values()))
    return sql
